In [ ]:
!pip install langchain-groq

In [160]:
pinecone_api_key = 'pcsk_3fmdnB_4wc5UXTGV1rcwr9ehmXzXNRyyx8vHg32kkwZtndUoyBTanKS7RruVi8WfjgHbwG'
groq_api_key = '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'


In [161]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_pinecone import PineconeEmbeddings

In [ ]:
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key,model='llama-3.3-70b-versatile')

In [163]:
from langchain.retrievers import PineconeHybridSearchRetriever
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()

pc = Pinecone(api_key=pinecone_api_key)
index_name = 'shldb2'
index = pc.Index(index_name)
embeddings = PineconeEmbeddings(api_key=pinecone_api_key, model = 'multilingual-e5-large')
vectorstore = PineconeVectorStore(index = index, embedding=embeddings)

try:
    retriever = vectorstore.as_retriever(
        search_type='similarity_score_threshold',
        search_kwargs={'k': 20, 'score_threshold': 0.7},
    )
    print("Retriever successfully created.")
except Exception as e:
    print(f"Error while creating retriever: {e}")

'''retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings, sparse_encoder=bm25_encoder, index=index, alpha = 0.5, top_k = 20,text_key="text"
)'''

Retriever successfully created.


'retriever = PineconeHybridSearchRetriever(\n    embeddings=embeddings, sparse_encoder=bm25_encoder, index=index, alpha = 0.5, top_k = 20,text_key="text"\n)'

In [164]:
query = "I am hiring for an analyst and wants applications to screen using Cognitive and personality tests, what options are available within 45 mins."

docs = retriever.invoke(query)

In [165]:
context = ""
for doc in docs:
    context += doc.page_content + " \n \n "


context

'Title : Phone Banker - Short Form \n URL : https://www.shl.com/solutions/products/product-catalog/view/phone-banker-short-form/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : This solution is for entry to mid-level financial institution positions. Candidates are asked to answer multiple choice questions to assess learning potential, dependability, persistence and planfulness, sales focus, customer focus, and working with information. Entry-Level, Mid-Professional, English (USA), Approximate Completion Time in minutes = 40 Test Type: Ability & Aptitude, Biodata & Situational Judgement, Personality & Behavior  Remote Testing \n\n  \n \n Title : Phone Banker - Short Form \n URL : https://www.shl.com/solutions/products/product-catalog/view/phone-banker-short-form/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : This solution is for entry to mid-level financial institution positions. Candidates are asked to answer multiple choice questions to assess learning potent

In [166]:
template = '''
List of assesments : \n
{context}
From the list of 20 assessments I've identified, select the top 1-5 options that best match technical skills (prioritized) followed by soft skills. Focus especially on programming languages and technical frameworks.

For each assessment, provide ONLY the following information in this exact format:
- Title: [exact title]
- URL: [complete URL]
- Remote Testing: [Yes/No]
- Adaptive/IRT: [Yes/No] 
- Duration: [specific time OR "Not specified" if not mentioned]
- Test Type: [extract this directly from the description after "test type"]

Important instructions:
1. Return exactly 1-5 assessments total
2. The duration listed must not exceed what's actually specified
3. Present ONLY the assessment list - no introduction, explanation, or conclusion
4. Sort by technical skill match first, then soft skill relevance
Answer for the following query: \n
{query} 
'''

In [167]:
prompt = PromptTemplate.from_template(template)

llm_chain = prompt | llm
response = llm_chain.invoke({"context": context, "query": query})
print(response.content)

- Title: Phone Banker - Short Form
- URL: https://www.shl.com/solutions/products/product-catalog/view/phone-banker-short-form/
- Remote Testing: Yes
- Adaptive/IRT: No
- Duration: 40
- Test Type: Ability & Aptitude, Biodata & Situational Judgement, Personality & Behavior

- Title: Bank Collections Agent - Short Form
- URL: https://www.shl.com/solutions/products/product-catalog/view/bank-collections-agent-short-form/
- Remote Testing: Yes
- Adaptive/IRT: No
- Duration: 45
- Test Type: Ability & Aptitude, Biodata & Situational Judgement, Personality & Behavior

- Title: Customer Service Phone Solution
- URL: https://www.shl.com/solutions/products/product-catalog/view/customer-service-phone-solution/
- Remote Testing: Yes
- Adaptive/IRT: No
- Duration: 30
- Test Type: Biodata & Situational Judgement, Personality & Behavior, Simulations

- Title: Contact Centre Agent Solution - UK
- URL: https://www.shl.com/solutions/products/product-catalog/view/contact-centre-agent-solution-uk/
- Remote 

'Title : Python (New) \n URL : https://www.shl.com/solutions/products/product-catalog/view/python-new/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Multi-choice test that measures the knowledge of Python programming, databases, modules and library. Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = 11 Test Type: Knowledge & Skills  Remote Testing \n\n  \n \n Title : Financial Professional - Short Form \n URL : https://www.shl.com/solutions/products/product-catalog/view/financial-professional-short-form/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : This solution is for mid-level financial institution positions that require Series 6/7 certification. Candidates answer multiple choice questions to measure financial professional aptitude, learning potential, achievement orientation, conscientiousness, persistence and planfulness, self leadership, interpersonal leadership, sales focus and customer focus. Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = 35 Test Type: Ability & Aptitude, Biodata & Situational Judgement, Personality & Behavior  Remote Testing \n\n  \n \n Title : JavaScript (New) \n URL : https://www.shl.com/solutions/products/product-catalog/view/javascript-new/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Multi-choice test that measures knowledge of programming in the JavaScript language and its application in front-end development. Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = 9 Test Type: Knowledge & Skills  Remote Testing \n\n  \n \n Title : Smart Interview On Demand \n URL : https://www.shl.com/solutions/products/product-catalog/view/smart-interview-on-demand/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Smart Interview On Demand is arecorded(asynchronous) video interview tool, that introduces you to each person behind the resume with a short-recorded video of them explaining why they would be an awesome addition to your team. A streamlined, minimum bias screening to ensure you shortlist top talent faster and find those hidden gems you may have otherwise missed. Availability Theadministrator and evaluator interfaceis available in US English and Simplified Chinese only, as are the standard question banks. However, custom questions can be added or recorded in any language. Theparticipant interfaceis available in: Arabic, Brazilian Portuguese, Bulgarian, Canadian French, Chinese, Croatian, Czech, Danish, Dutch, Dutch (Belgium), Estonian, Finnish, French, German, Greek, Hindi, Hungarian, Italian, Indonesian, Japanese, Korean, Latvian, Lithuanian, Malaysian, Mexican Spanish, Norwegian, Polish, Portuguese, Romanian, Russian, Serbian, Slovak, Slovenian, Spanish, Swedish, Taiwanese Chinese, Thai, Turkish, UK English, Ukrainian, US English, Vietnamese.Your use of this assessment product may be subject to New York City Law 144 (Regulation of the Use of Automated Employment Decision Tools) (dated July 5, 2023). Compliance with Law 144 is your responsibility. Read more on https://www.shl.com/legal/shl-us-regulatory-compliance/ Entry-Level, Supervisor, Manager, Front Line Manager, General Population, Graduate, Mid-Professional, Professional Individual Contributor, English (USA), Chinese Simplified, Approximate Completion Time in minutes = Variable Test Type: Personality & Behavior  Remote Testing \n\n  \n \n Title : Phone Banker - Short Form \n URL : https://www.shl.com/solutions/products/product-catalog/view/phone-banker-short-form/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : This solution is for entry to mid-level financial institution positions. Candidates are asked to answer multiple choice questions to assess learning potential, dependability, persistence and planfulness, sales focus, customer focus, and working with information. Entry-Level, Mid-Professional, English (USA), Approximate Completion Time in minutes = 40 Test Type: Ability & Aptitude, Biodata & Situational Judgement, Personality & Behavior  Remote Testing \n\n  \n \n Title : Phone Banker - Short Form \n URL : https://www.shl.com/solutions/products/product-catalog/view/phone-banker-short-form/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : This solution is for entry to mid-level financial institution positions. Candidates are asked to answer multiple choice questions to assess learning potential, dependability, persistence and planfulness, sales focus, customer focus, and working with information. Entry-Level, Mid-Professional, English (USA), Approximate Completion Time in minutes = 40 Test Type: Ability & Aptitude, Biodata & Situational Judgement, Personality & Behavior  Remote Testing \n\n  \n \n Title : AI Skills \n URL : https://www.shl.com/solutions/products/product-catalog/view/ai-skills/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : The AI Skills assessment measures the skills that help candidates successfully leverage AI in their work. General Population, English (USA), Approximate Completion Time in minutes = 16 Test Type: Personality & Behavior  Remote Testing \n\n  \n \n Title : Automata Pro (New) \n URL : https://www.shl.com/solutions/products/product-catalog/view/automata-pro-new/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : An AI-powered coding simulation assessment that evaluates candidate’s programming ability. Offers a familiar IDE environment available in over 40 different programming languages and tests candidates using real-world coding problems.Your use of this assessment product may be subject to New York City Law 144 (Regulation of the Use of Automated Employment Decision Tools) (dated July 5, 2023). Compliance with Law 144 is your responsibility. Read more on https://www.shl.com/legal/shl-us-regulatory-compliance/ Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = max 60 Test Type: Simulations  Remote Testing \n\n  \n \n Title : SQL (New) \n URL : https://www.shl.com/solutions/products/product-catalog/view/sql-new/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Multi-choice test that measures the knowledge of SQL queries, data manipulation and transaction processing. Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = 9 Test Type: Knowledge & Skills  Remote Testing \n\n  \n \n Title : Automata Selenium \n URL : https://www.shl.com/solutions/products/product-catalog/view/automata-selenium/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : A coding simulation assessment that evaluates the ability to conduct tasks related to automation testing using Selenium scripts. Entry-Level, Graduate, Manager, Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = 60 Test Type: Simulations  Remote Testing \n\n  \n \n Title : Automata (New) \n URL : https://www.shl.com/solutions/products/product-catalog/view/automata-new/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : An AI-powered coding simulation assessment that evaluates candidate’s programming ability. Offers a familiar IDE environment available in over 40 different programming languages and tests candidates using real-world coding problems.Your use of this assessment product may be subject to New York City Law 144 (Regulation of the Use of Automated Employment Decision Tools) (dated July 5, 2023). Compliance with Law 144 is your responsibility. Read more on https://www.shl.com/legal/shl-us-regulatory-compliance/ Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = max 45 Test Type: Simulations  Remote Testing \n\n  \n \n Title : Sales Professional 7.0 Solution \n URL : https://www.shl.com/solutions/products/product-catalog/view/sales-professional-7-0-solution/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Our Sales Professional 7.0 solution is designed for entry to mid-level sales positions in which employees proactively sell products to customers, develop relationships with repeat customers, and have their pay and/or performance heavily based on sales revenue. Sample tasks for these jobs include, but are not limited to: promoting products to customers, persuading customers to buy products, and building ongoing customer relationships.Report Language Availability:English (USA). Mid-Professional, Professional Individual Contributor, English International, English (USA), French (Canada), Latin American Spanish, Approximate Completion Time in minutes = 21 Test Type: Competencies, Biodata & Situational Judgement  Remote Testing \n\n  \n \n Title : Visual Basic for Applications (New) \n URL : https://www.shl.com/solutions/products/product-catalog/view/visual-basic-for-applications-new/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Multi-choice test that measures the knowledge of VBA programming constructs, forms, controls and VBA security. Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = 7 Test Type: Knowledge & Skills  Remote Testing \n\n  \n \n Title : General Entry Level - All Industries 7.1 Solution \n URL : https://www.shl.com/solutions/products/product-catalog/view/general-entry-level-all-industries-7-1-solution/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Our General Entry Level – All Industries 7.1 solution is designed for entry-level positions across industries. This solution predicts the following types of behaviors foundational to all jobs: being on-time to work; following rules and policies; treating others respectfully; producing quality work; meeting goals; and effectively working alongside others. Solution can be used across all industries with entry-level positions. Example titles include, but are not limited to: Administrator, Clerk, Veterinary Assistant, Stocker, Dish Washer, Office Coordinator, Tour Guide, Cook, Assistant, Housekeeper, Waste Collector.“MCP Product: please note that you will need to select the correct package depending on which norm you intend to use. The package available for this product is Americas and International." Entry-Level, German, Norwegian, Finnish, French (Canada), Portuguese (Brazil), Latin American Spanish, English (USA), English International, Danish, Swedish, French, Italian, Approximate Completion Time in minutes = 19 Test Type: Biodata & Situational Judgement  Remote Testing \n\n  \n \n Title : Virtual Assessment and Development Centers \n URL : https://www.shl.com/solutions/products/product-catalog/view/virtual-assessment-and-development-centers/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : SHL’s innovative virtual assessment centers wow new hires or existing employees and show how you invest in your people. They are a highly engaging experience, inclusive and packed with predictive power to undercover potential. SHL’s virtual assessment and development center platform is packaged with features to support your candidates, assessors and center administrators. Deliver a full range of assessment center exercises, from inbox simulations and written analysis, through coaching and role-play with preparation materials and one to one live virtual interactions, to assigned or unassigned role group exercises. Exercises are available across a range of job levels and industry scenarios Want to find out more or book a demo? Visit https://www.shl.com/en/solutions/virtual/virtual-assessment-development-center/Language Availabilityfor the candidate and assessor simulation interface is shown in the list below.Language Availabilityfor the live interactions interface (Smart Interview Live) is currently: English US, Japanese, Simplified Chinese, Spanish, Portuguese, Brazilian Portuguese, French, Canadian French, Italian, Dutch, Latin American Spanish, Romanian, Indonesian, German, Greek. Director, Entry-Level, Executive, Front Line Manager, General Population, Graduate, Manager, Mid-Professional, Professional Individual Contributor, Supervisor, English (USA), English International, Italian, German, Portuguese, Spanish, Latin American Spanish, French, French (Canada), Dutch, Japanese, Indonesian, Greek, Romanian, Chinese Simplified, Arabic, Approximate Completion Time in minutes = N/A Test Type: Personality & Behavior  Remote Testing \n\n  \n \n Title : SQL Server Analysis Services (SSAS) (New) \n URL : https://www.shl.com/solutions/products/product-catalog/view/sql-server-analysis-services-%28ssas%29-%28new%29/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Multi-choice test that measures the knowledge of SSAS database, querying multidimensional analysis solutions, cube hierarchies, measures, dimensions, power BI, DAX, MDX, tabular model data access and security. Mid-Professional, Professional Individual Contributor, English (USA), Approximate Completion Time in minutes = 15 Test Type: Knowledge & Skills  Remote Testing \n\n  \n \n Title : OPQ Profile Report \n URL : https://www.shl.com/solutions/products/product-catalog/view/opq-profile-report/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : This is a graphical profile charts presenting results across the 32 OPQ scales. It is designed to be interpreted by OPQ trained users only.Please note this report is currently being updated on Talent Central as part of a reporting refresh project, and there may be visual differences when comparing Talent Central reports to ones generated on other assessment platforms. Differences are regarding the layout and display of the reports only and do not affect the underlying scoring. When viewing sample reports, please ensure you select the appropriate language and platform example.Where a sample report has no platform indicated, it is available on both the Talent Central and SODA platforms. Director, Executive, Front Line Manager, General Population, Graduate, Manager, Mid-Professional, Professional Individual Contributor, Supervisor, Arabic, Romanian, Slovak, Serbian, Chinese Simplified, Chinese Traditional, Danish, Flemish, Dutch, English International, Finnish, French (Belgium), French (Canada), French, German, Greek, Hungarian, Icelandic, Indonesian, Italian, Japanese, Korean, Norwegian, Polish, Portuguese (Brazil), Portuguese, Russian, Latin American Spanish, Spanish, Swedish, Thai, Turkish, English (USA), Czech, Estonian, Latvian, Lithuanian,  Remote Testing \n\n  \n \n Title : Technology Professional 8.0 Job Focused Assessment \n URL : https://www.shl.com/solutions/products/product-catalog/view/technology-professional-8-0-job-focused-assessment/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : The Technology Job Focused Assessment assesseskey behavioral attributesrequired for success in fast-paced, rapidly evolvingtechnology environments, providing an indication of the candidate’s longer termpotential. It is available as part of SHL’s Technology Hiring Solution. Entry-Level, Graduate, Mid-Professional, Professional Individual Contributor, German, Spanish, Latin American Spanish, French, Italian, Indonesian, Chinese Simplified, French (Canada), English International, English (USA), Approximate Completion Time in minutes = 16 Test Type: Competencies, Personality & Behavior  Remote Testing \n\n  \n \n Title : Professional 7.0 Solution \n URL : https://www.shl.com/solutions/products/product-catalog/view/professional-7-0-solution/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Our Professional 7.0 solution is designed for all non-managerial professional individual contributor positions. Sample tasks for these jobs include, but are not limited to: gathering requirements, analyzing data, managing projects, producing products or services.Report Language Availability:English (USA), Latin American Spanish. Mid-Professional, Professional Individual Contributor, English (USA), English International, French (Canada), Latin American Spanish, Approximate Completion Time in minutes = 33 Test Type: Biodata & Situational Judgement, Competencies  Remote Testing \n\n  \n \n Title : Sales Professional 7.1 (Americas) \n URL : https://www.shl.com/solutions/products/product-catalog/view/sales-professional-7-1-%28americas%29/ \n Remote Testing : Yes \n Adaptive : No \n\n Description : Our Sales Professional 7.1 solution is designed for entry to mid-level sales positions in which employees proactively sell products to customers, develop relationships with repeat customers, and have their pay and/or performance heavily based on sales revenue. Sample tasks for these jobs include, but are not limited to: promoting products to customers, persuading customers to buy products, and building ongoing customer relationships. Professional Individual Contributor, Mid-Professional, English (USA), Portuguese (Brazil), Latin American Spanish, French (Canada), Approximate Completion Time in minutes = 21 Test Type: Biodata & Situational Judgement, Competencies  Remote Testing \n\n  \n \n '